## Table Of Contents
 1. [Analisi esplorativa con `PCA`, `ICA`, `t-SNE`](#esplorativa)
 2. [Modello multinomiale](#multinomiale)<br>
     2.1 [Modello multinomiale penalizzato](#multinomiale-pen)
 3. [Analisi discriminante](#lda-qda)<br>
     3.1 [Analisi discriminante penalizzata](#da-pen)
 4. [Alberi di regressione](#tree)<br>
     4.1 [Alberi di regressione penalizzati](#tree-penalized)<br>

END. [Salvataggio dei dati](#save)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Funzioni base:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import tqdm

# Font di LaTeX
# from matplotlib import rc

# Scikit-Learn
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.decomposition import PCA, FastICA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz
# Funzioni custom
from funzioni import AbsMeanVarDeriv, Whiten,ScatterGroup, MatriceConfusione, indice_gini, DistGroup
from funzioni import tasso_errata_classificazione, grafico_metrica_iperparametro, grafico_metrica_iperparametri
from funzioni.grafici import grafico_importanza_variabili

# 1. Analisi esplorativa con `PCA`, `ICA`, `t-SNE` <a id=esplorativa> </a>

In [ ]:
PATH_DATA = './PhonePi/data/'
FIG_PATH = './figure/'
DIR = [os.path.join(PATH_DATA, o) for o in os.listdir(PATH_DATA) 
                    if os.path.isdir(os.path.join(PATH_DATA,o))]
tipo=[(dir.split("/")[-1]).split(".")[0] for dir in DIR]
tipo=[dir.split("-")[0] for dir in tipo]

In [ ]:
y = pd.read_pickle("y-2s.pkl")
X_train = pd.read_pickle("X_train.pkl")
y_train = pd.read_pickle("y_train.pkl")
X_val = pd.read_pickle("X_val.pkl")
y_val = pd.read_pickle("y_val.pkl")

In [ ]:
for col in X_train.columns:
    DistGroup(X_train[col], grp=y, leg_loc = (1.4, 0.8),
              palette="colorblind", file = FIG_PATH + col + ".png")

In [ ]:
# Sbiancamento dei dati
XWh = Whiten().fit_transform(X_train)

pca = PCA(n_components=2, random_state=42)
XPCA = pca.fit_transform(XWh)

ica = FastICA(n_components=2, random_state=42)
XICA = ica.fit_transform(XWh)

tsne = TSNE(n_components=2, random_state=42)
XTSNE = tsne.fit_transform(XWh)

In [ ]:
for title,dat in zip(["PCA","ICA","t-SNE"], [XPCA, XICA, XTSNE]):
    fig, ax = ScatterGroup(pd.DataFrame(dat, columns=["Prima componente", "Seconda componente"]),
                       grp=y, palette="colorblind")
    fig.set_figwidth(11)
    fig.set_figheight(6)
    ax.set_title(title)
    plt.legend(bbox_to_anchor=(1,0.7))
    plt.savefig(FIG_PATH+title+".png", bbox_inches="tight", dpi=180)

# 2. Modello multinomiale <a id=multinomiale> </a>

In [ ]:
glmMult = LogisticRegression(penalty="l2", C=float("inf"), random_state=42, multi_class="multinomial",
                             solver="newton-cg", max_iter=300)
fit = glmMult.fit(X_train, y_train)
y_pred = fit.predict(X_val)

acc_mn = 100*accuracy_score(y_val, y_pred)
print("Regressione multinomiale: {:.1f}% di accuratezza".format(acc_mn))
MatriceConfusione(y_val, y_pred, nome_immagine=FIG_PATH+"confusionMatrix-Mn.png", title="Multinomiale")

## 2.1 Modello multinomiale penalizzato <a id=multinomiale-pen></a>

In [ ]:
fit.classes_

In [ ]:
lab = y.unique()
lab.sort()
print(lab)

In [ ]:
def PesiShakeMn(val):
    '''
    Funzione per modificare i pesi nel modello multinomiale
    
    Parameters
    ----------
    Modello : Funzione di classificazione (LDA o QDA)
    
    val : array di valori per cui dividere il peso base della classe shake
    '''
    iteraz = []
    lista_acc=[]
    lista_veri_positivi=[]
    peso_shake=[]
    for i in tqdm.tqdm(val):
        iteraz.append(i)
        weights = y_train.value_counts()/y_train.value_counts().sum()
        weights["shake"]/=i
        weights/=weights.sum()
        w = {ind:weights[ind] for ind in weights.index}
        peso_shake.append(weights["shake"])
        model = LogisticRegression(penalty="l2", C=float("inf"), random_state=42, 
                               multi_class="multinomial",solver="newton-cg", max_iter=100, class_weight=w)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        lista_acc.append(100*accuracy_score(y_val, y_pred))
        lista_veri_positivi.append(100*(confusion_matrix(y_val, y_pred, labels=lab)[-1,-1]/confusion_matrix(y_val, y_pred,labels=lab)[:,-1].sum()))
        
    return peso_shake, lista_acc, lista_veri_positivi

In [ ]:
import warnings
warnings.filterwarnings("ignore")
pesi = np.arange(1, 13, 0.5)
peso_mn, lista_acc_mn, lista_veri_positivi_mn = PesiShakeMn(val=pesi)

In [ ]:
lista_acc_mn[-9]

In [ ]:
plt.plot(lista_acc_mn, label = "accuratezza")
plt.plot(lista_veri_positivi_mn, label = "shake corretti")
plt.title("Multinomiale")
plt.xlabel("iterazione")
plt.ylabel("%")
plt.legend(bbox_to_anchor=(1.31,0.7))

plt.show()

In [ ]:
i = pesi[lista_veri_positivi_mn.index(100)]
weights = y_train.value_counts()/y_train.value_counts().sum()
weights["shake"]/=i
weights/=weights.sum()
w = {ind:weights[ind] for ind in weights.index}
model = LogisticRegression(penalty="l2", C=float("inf"), random_state=42, 
                               multi_class="multinomial",solver="newton-cg", max_iter=100, class_weight=w)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
acc_mn_pen = 100*accuracy_score(y_val, y_pred)
print("Accuratezza Multinomiale penalizzata: {:.1f}%".format(acc_mn_pen))
MatriceConfusione(y_val, y_pred,nome_immagine=FIG_PATH+"confusionMatrix-Mn-penalizzata", title="Multinomiale pesata")

In [ ]:
#-------------------- TODO ---------------------#
# Prendere il peso migliore per la multinomiale #
#-----------------------------------------------#
#w = ...


#glmMult = LogisticRegression(penalty="l2", C=float("inf"), random_state=42, multi_class="multinomial",
#                              solver="newton-cg", max_iter=300, class_weight=w)
#fit = glmMult.fit(X_train, y_train)
# y_pred = fit.predict(X_val)

# acc_mn_pen = 100*accuracy_score(y_val, y_pred)
# print("Regressione multinomiale penalizzata: {:.1f}% di accuratezza".format(acc_mn_pen))
# MatriceConfusione(y_val, y_pred)
# plt.savefig(FIG_PATH+"confusionMatrix-Mn-penalizzata.png", dpi=300, bbox_inches="tight")

# 3. Analisi discriminante lineare e quadratica <a id=lda-qda> </a>

In [ ]:
lda = LinearDiscriminantAnalysis()
X_lda=lda.fit(X_train, y_train)
y_pred_lda = lda.predict(X_val)
acc_lda = 100*accuracy_score(y_val, y_pred_lda)
print("Accuratezza LDA: {:.1f}%".format(acc_lda))
MatriceConfusione(y_val, y_pred_lda,nome_immagine=FIG_PATH+"confusionMatrix-LDA", title="LDA")

In [ ]:
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)
y_pred_qda = qda.predict(X_val)
acc_qda = 100*accuracy_score(y_val, y_pred_qda)
print("Accuratezza QDA: {:.1f}%".format(acc_qda))
MatriceConfusione(y_val, y_pred_qda,nome_immagine=FIG_PATH+"confusionMatrix-QDA", title="QDA")

## 3.1 Analisi discriminante penalizzata <a id=da-pen> </a>

In [ ]:
print(lda.classes_)

In [ ]:
def PesiShakeDA(Modello, val):
    '''
    Funzione per modificare i pesi delle analisi discriminanti lineare e quadratica.
    
    Parameters
    ----------
    Modello : Funzione di classificazione (LDA o QDA)
    
    val : array di valori per cui dividere il peso base della classe shake
    '''

    lista_acc=[]
    lista_veri_positivi=[]
    w=[]
    for i in val:
        weights = np.array(y_train.value_counts()/y_train.value_counts().sum())
        weights[-1] /= i
        weights = weights/weights.sum()
        w.append(weights[-1])
        model = Modello(priors=weights)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        lista_acc.append(100*accuracy_score(y_val, y_pred))
        lista_veri_positivi.append(100*(confusion_matrix(y_val, y_pred, labels=lab)[-1,-1]/confusion_matrix(y_val, y_pred,labels=lab)[:,-1].sum()))
        
    return w, lista_acc, lista_veri_positivi

### LDA e QDA penalizzate

In [ ]:
# in pratica per pesi crescenti assegnati alla classe dello shake si nota che la percentuale di veri positivi aumenta
# e l'accuratezza ovviamente diminuisce. Se vogliamo che l'errore di classificare shake qualcosa che non è shake sia nullo
# (o <0.01) scegliamo il primo peso che mi porta ad avere veri_positivi uguale a 100 (o 99) in modo da scegliere 
# il modello con l'accuratezza migliore fra quelli che non sbagliano lo shake

val = np.arange(1.1, 12, 0.1) # lista di divisori del peso per la classe

pesi_lda, lista_acc_lda, lista_veri_positivi_lda = PesiShakeDA(LinearDiscriminantAnalysis, val)
plt.plot(pesi_lda, lista_acc_lda, label = "accuratezza")
plt.plot(pesi_lda, lista_veri_positivi_lda, label = "shake corretti")
plt.title("LDA")
plt.xlabel("iterazione")
plt.ylabel("%")
plt.legend(bbox_to_anchor=(1.34,0.7))
plt.show()

pesi_qda, lista_acc_qda, lista_veri_positivi_qda = PesiShakeDA(QuadraticDiscriminantAnalysis, val)
plt.plot(pesi_qda, lista_acc_qda, label = "accuratezza")
plt.plot(pesi_qda, lista_veri_positivi_qda, label = "shake corretti")
plt.title("QDA")
plt.xlabel("peso shake")
plt.ylabel("%")
plt.legend(bbox_to_anchor=(1.34,0.7))
plt.savefig(FIG_PATH + "acc-vs-veripos-qda", bbox_inches="tight", dpi=300)
plt.show()

In [ ]:
plt.plot(pesi_qda)
plt.xlabel("iterazione")
plt.ylabel("peso")
plt.savefig(FIG_PATH + "andamento-pesi.png", bbox_inches="tight", dpi=300)

In [ ]:
lista_veri_positivi_qda[-1]

In [ ]:
lista_veri_positivi_lda=np.array(lista_veri_positivi_lda)
num_iterazione_lda=np.min(np.where(lista_veri_positivi_lda>99))
peso_shake_lda=np.arange(1.1,12,0.1)[num_iterazione_lda]
peso_shake_lda

In [ ]:
lista_veri_positivi_qda=np.array(lista_veri_positivi_qda)
num_iterazione_qda=np.min(np.where(lista_veri_positivi_qda>97))
peso_shake_qda=np.arange(1.1,12,0.1)[num_iterazione_qda]
peso_shake_qda

In [ ]:
weights = np.array(y_train.value_counts()/y_train.value_counts().sum())
weights[-1] /= peso_shake_lda
weights = weights/weights.sum()
lda = LinearDiscriminantAnalysis(priors=weights)
X_lda=lda.fit(X_train, y_train)
y_pred_lda = lda.predict(X_val)
acc_lda_pen = 100*accuracy_score(y_val, y_pred_lda)
print("Accuratezza LDA penalizzata: {:.1f}%".format(acc_lda_pen))
MatriceConfusione(y_val, y_pred_lda, nome_immagine=FIG_PATH+"confusionMatrix-LDA-penalizzata", title="LDA pesata")

In [ ]:
weights = np.array(y_train.value_counts()/y_train.value_counts().sum())
weights[-1] /= peso_shake_qda
weights = weights/weights.sum()
qda = QuadraticDiscriminantAnalysis(priors=weights)
qda.fit(X_train, y_train)
y_pred_qda = qda.predict(X_val)
acc_qda_pen = 100*accuracy_score(y_val, y_pred_qda)
print("Accuratezza QDA penalizzata: {:.1f}%".format(acc_qda_pen))
MatriceConfusione(y_val, y_pred_qda,nome_immagine=FIG_PATH+"confusionMatrix-QDA-penalizzata", title="QDA pesata")

# 4. Alberi di regressione <a id=tree> </a>

In [ ]:
# Albero stimato sul training, senza vincoli (albero completo)
dtc = DecisionTreeClassifier(random_state=42)
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_val)
acc_dtcFull = 100*accuracy_score(y_val, y_pred)

print("Accuratezza DecisionTreeClassifier(): {:.2f}%".format(acc_dtcFull))
MatriceConfusione(y_val, y_pred, FIG_PATH+"confusionMatrix-Tree", title="Albero di regressione completo")
plt.show()

In [ ]:
maxDepth = dtc.tree_.max_depth
minObs = len(X_train) // 2
print("Profondità dell'albero allenato senza restrizioni: {}".format(maxDepth))
print("Massimo numero minimo di osservazioni in una foglia: {}".format(minObs))

In [ ]:
param_grid = ParameterGrid({
    'max_depth': np.arange(1, dtc.tree_.max_depth+1),
    'min_samples_leaf': 2 ** np.arange(int(np.log2(minObs) + 1)),
})
print(param_grid.param_grid)

In [ ]:
risultati = []

for params in tqdm.tqdm(param_grid):
    dtc = DecisionTreeClassifier(random_state=42, **params)
    dtc.fit(X_train, y_train)
    y_pred = dtc.predict(X_val)
    params["accuracy_score"] = accuracy_score(y_val, y_pred)
    risultati.append(params)

risultati = pd.DataFrame(risultati).sort_values(["accuracy_score", "max_depth"], ascending=[False, True])
risultati.reset_index(drop=True, inplace=True)
print("Primi 5:")
display(risultati.head())

print("Ultimi 5:")
risultati.tail()

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(121)
grafico_metrica_iperparametro(risultati, "max_depth", "accuracy_score", alpha=0.5)

plt.subplot(122)
grafico_metrica_iperparametro(risultati, "min_samples_leaf", "accuracy_score", alpha=0.5)
plt.xscale("log", basex=2)

plt.show()

In [ ]:
plt.figure(figsize=(12, 6))

grafico_metrica_iperparametri(risultati, "max_depth", "min_samples_leaf", "accuracy_score")
plt.yscale("log", basey=2)
plt.savefig(FIG_PATH + "iperparametri-Tree.png", dpi=200)
plt.show()

In [ ]:
max_depth = risultati.loc[0, "max_depth"]
min_samples_leaf = risultati.loc[0, "min_samples_leaf"]

dtcTun = DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=min_samples_leaf, random_state=42)
dtcTun.fit(X_train, y_train)

y_pred = dtcTun.predict(X_val)
acc_dtcTun = 100*accuracy_score(y_val, y_pred)
print("profondità ottimale:",max_depth)
print("numero ottimale minimo di unità per foglia:",min_samples_leaf)

In [ ]:
print("Accuratezza DecisionTreeClassifier(): {:.1f}%".format(acc_dtcFull))
print("Accuratezza DecisionTreeClassifier(max_depth={}, min_samples_leaf={}): {:.1f}%".format(
    max_depth, min_samples_leaf, acc_dtcTun))
print(confusion_matrix(y_val, y_pred))

MatriceConfusione(y_val, y_pred, nome_immagine=FIG_PATH + "confusionMatrix-Tree.png",
                  title="Albero di classificazione ottimizzato")

In [ ]:
importanze = dtcTun.feature_importances_
variabili = espl.columns

grafico_importanza_variabili(importanze, variabili)
plt.savefig(FIG_PATH + "importance-Tree.png", dpi=300, bbox_inches="tight")
plt.show()

## 4.1 Alberi di regressione penalizzati  <a id=tree-penalized> </a>

In [ ]:
dtc.classes_

In [ ]:
i=20
weights = y_train.value_counts()/y_train.value_counts().sum()
weights["shake"]/=i
weights/=weights.sum()
w = {ind:weights[ind] for ind in weights.index}

In [ ]:
dtc = DecisionTreeClassifier(random_state=42,
                             class_weight=w)

dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_val)
acc_dtc_pen = 100*accuracy_score(y_val, y_pred)
print("Accuratezza DecisionTreeClassifier(): {:.1f}%".format(acc_dtc_pen))
MatriceConfusione(y_val, y_pred)

# END. Salvataggio dei risultati <a id=save> </a>

## Tabella accuracy in file LaTeX

In [ ]:
def DataFrameToTable(df, file, cap="", lab="", digits="%.2f"):
    '''
    Funzione che stampa un dataframe in tabella di LaTeX.
    
    Parametri
    ---------
    df : DataFrame
        DataFrame da stampare in tabella
        
    file : string
        Percorso completo al file in cui scrivere la tabella
        
    cap : string
        Caption per la tabella
    
    lab : string
        Label della tabella
    
    '''

    tableEnvBegin = "\\begin{table}[H]\n\\centering\n"
    tableEnvEnd = "\\end{table}"
    c = 1 if cap else 0
    l = 1 if lab else 0
    caption="\\caption{"*c + cap + "}\n"*c
    label="\\label{"*l + lab + "}\n"*l
        
    with open(file, mode="w") as f:
        f.write(tableEnvBegin + caption +
                   df.to_latex(index=False, float_format=digits, column_format="c"*len(df.columns)) +
                   label + tableEnvEnd)
    

In [ ]:
accuracy = pd.DataFrame([["Multinomiale", acc_mn, 100*77/(7+1+77) ],
             ["LDA", acc_lda, 100*53/(1+10+2+53)],
             ["QDA", acc_qda, 100*77/(1+2+6+2+77)],
             ["Albero decisionale", acc_dtcFull, 100*74/(1+10+4+73)],
             ["Albero decisionale ottimizzato", acc_dtcTun, 100*71/(1+10+3+71)]], columns=["Modello", "Accuratezza %", "Shake corretti %"])

accuracy.sort_values("Accuratezza %", inplace=True, ascending=False)


In [ ]:
DataFrameToTable(accuracy, file = "./relazione/tex/accuracy-table.tex",
                cap = "Accuratezza per i modelli adattati sull'insieme di convalida.",
                lab = "tab:acc", )

In [ ]:
accuracy = pd.DataFrame([["Multinomiale", acc_mn_pen],
                         ["LDA", acc_lda_pen],
                         ["QDA", acc_qda_pen]],
                        columns=["Modello", "Accuratezza %"])
accuracy.sort_values("Accuratezza %", inplace=True, ascending=False)

DataFrameToTable(accuracy, file = "./relazione/tex/accuracy-table-pen.tex",
                 cap = "Accuratezza per i modelli adattati sull'insieme di convalida.",
                 lab = "tab:acc_pen")

# Insieme di test

In [ ]:
X_test = pd.read_pickle("X_test.pkl")
y_test = pd.read_pickle("y_test.pkl")

In [ ]:
model = LogisticRegression(penalty="l2", C=float("inf"), random_state=42, 
                               multi_class="multinomial",solver="newton-cg", max_iter=300)
model.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))
y_pred = model.predict(X_test)
acc_mn_test = 100*accuracy_score(y_test, y_pred)
print("Accuratezza Multinomiale sull'insieme di test: {:.1f}%".format(acc_mn_test))
MatriceConfusione(y_test, y_pred,nome_immagine=FIG_PATH+"confusionMatrix-Mn-test"
                 , title = "Multinomiale")


In [ ]:
weights = np.array(y_train.value_counts()/y_train.value_counts().sum())
weights[-1] /= peso_shake_qda
weights = weights/weights.sum()
qda = QuadraticDiscriminantAnalysis(priors=weights)
qda.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))
y_pred_qda = qda.predict(X_test)
acc_qda_pen = 100*accuracy_score(y_test, y_pred_qda)
print("Accuratezza QDA pesata sull'insieme di test: {:.1f}%".format(acc_qda_pen))
MatriceConfusione(y_test, y_pred_qda,nome_immagine=FIG_PATH+"confusionMatrix-QDA-penalizzata-test",
                 title="QDA pesata")

## Grafici delle esplicative di esempio

In [ ]:
X = pd.read_pickle("./X-2s.pkl")
X.drop(columns="user", inplace=True)

In [ ]:
X_shake = X[y == "shake"]
X_scale = X[y == "camminata"]

In [ ]:
X.iloc[0, :]

In [ ]:
t = np.arange(0, 150)
for i in range(5):
    plt.title("shake", fontsize=13)
    plt.ylabel(r"$\|a_j\|\;(\mathrm{ m/s^2})$", fontsize=13, rotation=0, labelpad=35)
    plt.xlabel("indice", fontsize=13, labelpad=15)
    plt.plot(t, X_shake.iloc[i, :], color="darkblue", alpha=0.55)
    plt.savefig(FIG_PATH + "shake.png", dpi=300, bbox_inches="tight")

In [ ]:
t = np.arange(0, 150)
for i in range(5):
    plt.title("camminata", fontsize=13)
    plt.ylabel(r"$\|a_j\|\;(\mathrm{ m/s^2})$", fontsize=13, rotation=0, labelpad=35)
    plt.xlabel("indice", fontsize=13, labelpad=15)
    plt.plot(t, X_scale.iloc[i, :], color="darkblue", alpha=0.55)
    plt.savefig(FIG_PATH + "camminata.png", dpi=300, bbox_inches="tight")

### Crop delle immagini

In [ ]:
import platform
if platform.system() == "Linux":
    !mogrify -trim ./figure/*.png